In [3]:
import pandas as pd
import numpy as np 
from gpxutils import parse_gpx 
import matplotlib.pyplot as plt
%matplotlib inline

# Analysis of Cycling Data

We are provided with four files containing recordings of cycling activities that include GPS location data as
well as some measurements related to cycling performace like heart rate and power.  The goal is to perform
some exploration and analysis of this data. 

The data represents four races.  Two are time trials where the rider rides alone on a set course.  Two are 
road races where the rider rides with a peleton.  All were held on the same course but the road races include
two laps where the time trials include just one. 

Questions to explore with the data:
* What is the overall distance travelled for each of the rides? What are the average speeds etc.  Provide a summary for each ride.
* Compare the range of speeds for each ride, are time trials faster than road races? 
* Compare the speeds achieved in the two time trials (three years apart).  As well as looking at the averages, can you see where in the ride one or the other is faster.  
* From the elevation_gain field you can see whether the rider is _climbing_ , _descending_ or on the _flat_.   Use this to calculate the average speeds in those three cases (climbing, flat or descending).  Note that _flat_ might not be zero elevation_gain but might allow for slight climbs and falls.  

For time varying data like this it is often useful to _smooth_ the data using eg. a [rolling mean](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.rolling_mean.html).  You might want to experiment with smoothing in some of your analysis (not required but may be of interest).

## Description of Fields

* _index_ is a datetime showing the time that the observation was made (I wasn't riding at night, this is converted to UTC)
* __latitude, longitude, elevation__ from the GPS, the position of the rider at each timepoint, elevation in m
* __temperature__ the current ambient temperature in degrees celcius
* __power__ the power being generated by the rider in Watts
* __cadence__ the rotational speed of the pedals in revolutions per minute
* __hr__ heart rate in beats per minute
* __elevation_gain__ the change in elevation in m between two observations
* __distance__ distance travelled between observations in km
* __speed__ speed measured in km/h

You are provided with code in [gpxutils.py](gpxutils.py) to read the GPX XML format files that are exported by cycling computers and applications.  The sample files were exported from [Strava](https://strava.com/) and represent four races by Steve Cassidy.


In [4]:
# read the four data files
rr_2016 = parse_gpx('files/Calga_RR_2016.gpx')
tt_2016 = parse_gpx('files/Calga_TT_2016.gpx')
rr_2019 = parse_gpx('files/Calga_RR_2019.gpx')
tt_2019 = parse_gpx('files/Calga_TT_2019.gpx')

In [5]:
rr_2016.head()

,latitude,longitude,elevation,temperature,power,cadence,hr,distance,elevation_gain,speed,timedelta
2016-05-14 04:02:41+00:00,-33.415561,151.222303,208.6,29.0,0.0,40.0,102.0,0.000000,0.0,0.000000,NaN
2016-05-14 04:02:42+00:00,-33.415534,151.222289,208.6,29.0,0.0,40.0,102.0,0.003271,0.0,11.777020,1.0
2016-05-14 04:02:46+00:00,-33.415398,151.222180,208.6,29.0,0.0,40.0,103.0,0.018194,0.0,16.375033,4.0
2016-05-14 04:02:49+00:00,-33.415264,151.222077,208.6,29.0,0.0,55.0,106.0,0.017703,0.0,21.243901,3.0
2016-05-14 04:02:51+00:00,-33.415160,151.222013,208.6,29.0,0.0,61.0,109.0,0.013001,0.0,23.401217,2.0


In [34]:
#What is the overall distance travelled for each of the rides? What are the average speeds etc.
#Provide a summary for each ride.
totalDistRR16 = rr_2016['distance'].sum()
totalDistTT16 = tt_2016['distance'].sum()
totalDistRR19 = rr_2019['distance'].sum()
totalDistTT19 = tt_2019['distance'].sum()

avgSpeedRR16 = rr_2016['speed'].mean()
avgSpeedTT16 = tt_2016['speed'].mean()
avgSpeedRR19 = rr_2019['speed'].mean()
avgSpeedTT19 = tt_2019['speed'].mean()

avgTimeRR16 = rr_2016['hr'].mean()
avgTimeTT16 = tt_2016['hr'].mean()
avgTimeRR19 = rr_2019['hr'].mean()
avgTimeTT19 = tt_2019['hr'].mean()

avgCadenceRR16 = rr_2016['cadence'].mean()
avgCadenceTT16 = tt_2016['cadence'].mean()
avgCadenceRR19 = rr_2019['cadence'].mean()
avgCadenceTT19 = tt_2019['cadence'].mean()

print("For the 2016 road race, the total distance travelled was " + str(totalDistRR16) +  
      ", the average speed was " + str(avgSpeedRR16) + ", the average time was " + str(avgTimeRR16) + 
     ", and the average cadence was " + str(avgCadenceRR16) + ".\n")

print("For the 2016 time trial, the total distance travelled was " + str(totalDistTT16) +  
      ", the average speed was " + str(avgSpeedTT16) + ", the average time was " + str(avgTimeTT16) + 
     ", and the average cadence was " + str(avgCadenceTT16) + ".\n")

print("For the 2019 road race, the total distance travelled was " + str(totalDistRR19) +  
      ", the average speed was " + str(avgSpeedRR19) + ", the average time was " + str(avgTimeRR19) + 
     ", and the average cadence was " + str(avgCadenceRR19) + ".\n")

print("For the 2019 time trial, the total distance travelled was " + str(totalDistTT19) +  
      ", the average speed was " + str(avgSpeedTT19) + ", the average time was " + str(avgTimeTT19) + 
     ", and the average cadence was " + str(avgCadenceTT19) + ".\n")

#Compare the range of speeds for each ride, are time trials faster than road races?
speedRangeRR16 = (rr_2016['speed'].max() - rr_2016['speed'].min())
speedRangeTT16 = (tt_2016['speed'].max() - tt_2016['speed'].min())
speedRangeRR19 = (rr_2019['speed'].max() - rr_2019['speed'].min())
speedRangeTT19 = (tt_2019['speed'].max() - tt_2019['speed'].min())

print("The range of speed for each ride is as follows:\nRoad Race 2016: " + str(speedRangeRR16) +
      "\nTime Trial 2016: " + str(speedRangeTT16) + 
      "\nRoad Race 2019: " + str(speedRangeRR19) + 
      "\nTime Trial 2019: " + str(speedRangeTT19) + 
      "\nIt can be inferred that time trials are not necessarily faster than road races; while the 2016 Time Trial was considerably faster, the 2019 Time Trial was notably slower.")


#Compare the speeds achieved in the two time trials (three years apart).
#As well as looking at the averages, can you see where in the ride one or the other is faster.


#From the elevation_gain field you can see whether the rider is climbing , descending or on the flat.
#Use this to calculate the average speeds in those three cases (climbing, flat or descending).
#Note that flat might not be zero elevation_gain but might allow for slight climbs and falls.



For the 2016 road race, the total distance travelled was 49.04858574628638, the average speed was 34.93308475482947, the average time was 158.39440113394755, and the average cadence was 65.98795180722891.

For the 2016 time trial, the total distance travelled was 24.80288703130808, the average speed was 33.52996304869014, the average time was 170.93964957819597, and the average cadence was 83.27709279688514.

For the 2019 road race, the total distance travelled was 51.78913253596059, the average speed was 33.87986137188044, the average time was 138.99854624750137, and the average cadence was 70.0049064146829.

For the 2019 time trial, the total distance travelled was 24.38014504376575, the average speed was 33.05782378815691, the average time was 152.74124293785312, and the average cadence was 89.97966101694915.

The range of speed for each ride is as follows:
Road Race 2016: 92.74903649913952
Time Trial 2016: 162.50576377606754
Road Race 2019: 70.37046868335639
Time Trial 2019: 63.300

## Challenge: Gear Usage

A modern race bike has up to 22 different gears with two chainrings on the front (attached to the pedals) and 10 or 11 at the back (attached to the wheel).   The ratio of the number of teeth on the front and rear cogs determines the distance travelled with one revolution of the pedals (often called __development__, measured in metres).  Low development is good for climbing hills while high development is for going fast downhill or in the final sprint. 

We have a measure of the number of rotations of the pedals per minute (__cadence__) and a measure of __speed__.  Using these two variables we should be able to derive a measure of __development__ which would effectivly tell us which gear the rider was using at the time.   Development will normally range between __2m__ and __10m__.  Due to errors in GPS and cadence measurements you will see many points outside this range and you should just discard them as outliers. 

Write code to calculate __development__ in _meters_ for each row in a ride.  Plot the result in a _histogram_ and compare the plots for the four rides.   Comment on what you observe in the histograms.



